## Open Websocket and produce Kafka msg

In [ ]:
from websockets.client import connect
import json
import os
import dotenv

from src.data_ingestion.kafka.base_producer import BaseProducer


dotenv.load_dotenv()

auth_msg = {
  "action": "auth",
  "key": os.environ["APCA_API_KEY_ID"],
  "secret": os.environ["APCA_API_SECRET_KEY"]
}
start_listen_msg= {
      "action": "listen",
      "data": {
        "streams": ["trade_updates"]
      }
    }
async with connect("wss://paper-api.alpaca.markets/stream") as websocket:
    await websocket.send(json.dumps(auth_msg))
    await websocket.send(json.dumps(start_listen_msg))
    data = await websocket.recv()

    async for message in websocket:
      print(data)
      BaseProducer(bootstrap_servers=os.environ["LOCAL_BOOSTRAP_SERVER"], sasl_mechanism="PLAIN").produce(json.loads(data))


In [9]:
import json
from websocket import create_connection
import websocket 

m = {
  "action": "listen",
  "data": {
    "streams": ["trade_updates"]
  }
}

websocket.enableTrace(False)
ws = create_connection("wss://paper-api.alpaca.markets/stream")
ws.send(json.dumps(auth_msg))
ws.send(json.dumps(m))

result = ws.recv()
print('Result: {}'.format(result))

Result: b'{"stream":"authorization","data":{"action":"authenticate","status":"authorized"}}'


## Get last news API and send to Kafka topic

In [10]:
import pandas as pd
import requests
import json 


url = "https://data.alpaca.markets/v1beta1/news"#?symbols=COIN,BTCUSD"

headers = {
    "accept": "application/json",
    "APCA-API-KEY-ID": os.environ["APCA_API_KEY_ID"],
    "APCA-API-SECRET-KEY": os.environ["APCA_API_SECRET_KEY"]
}

response = requests.get(url, headers=headers)

news_response = json.loads(response.text)

In [11]:
news_response_df = pd.DataFrame(news_response["news"])
news_response_df.head(2)

,author,content,created_at,headline,id,images,source,summary,symbols,updated_at,url
0,Benzinga Neuro,,2024-04-18T12:17:53Z,Analysts Forecast 22% Surge For This 'Dividend...,38312273,"[{'size': 'large', 'url': 'https://cdn.benzing...",benzinga,With geopolitical tensions and high inflation ...,[CAT],2024-04-18T12:17:54Z,https://www.benzinga.com/markets/equities/24/0...
1,Benzinga Newsdesk,,2024-04-18T12:17:37Z,Home BancShares Q1 Adj EPS $0.49 Beats $0.46 E...,38312271,[],benzinga,,[HOMB],2024-04-18T12:17:37Z,https://www.benzinga.com/news/earnings/24/04/3...


In [ ]:
import dotenv
import os

from src.data_ingestion.kafka.base_producer import BaseProducer

dotenv.load_dotenv()

producer = BaseProducer(bootstrap_servers=os.environ["LOCAL_BOOSTRAP_SERVER"], sasl_mechanism="PLAIN")

In [ ]:
producer.produce("a", "a", "a")

for i in news_response_df.to_dict(orient='records'):
    producer.produce("news-topic", "a", i)

In [ ]:
from kafka import KafkaConsumer

consumer = KafkaConsumer('a', consumer_timeout_ms=1, auto_offset_reset='earliest', group_id='news-persit-data-lake', bootstrap_servers=os.environ["LOCAL_BOOSTRAP_SERVER"])

In [ ]:
for i in consumer:
    print(i)

In [ ]:
from src.data_ingestion.kafka.base_consumer import BaseConsumer


consumer = BaseConsumer()

In [ ]:
consumer.consume_next()